<a href="https://colab.research.google.com/github/snpushpi/Covid19_Contact_tracing_repo_beta/blob/master/VRU_DatasetFinal_No_Fake_path_contact_tracing_same_hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [1]:
import pandas as pd
import os

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Processed1_Dataset.csv to Processed1_Dataset.csv


In [4]:
import io

In [7]:
df = pd.read_csv(io.BytesIO(uploaded['Processed1_Dataset.csv']))

In [8]:
len(df[df.ID==0])

460

In [10]:
df[:11]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,x,y,ID,modified_x,modified_y
0,0,0,0,0.00,-1.43164,2.05182,0,-1.0,2.0
1,1,1,1,0.02,-1.41183,2.03714,0,-1.0,2.0
2,2,2,2,0.04,-1.39942,2.01728,0,-1.0,2.0
3,3,3,3,0.06,-1.38803,1.99488,0,-1.0,2.0
4,4,4,4,0.08,-1.37095,1.97715,0,-1.0,2.0
5,5,5,5,0.10,-1.35568,1.95657,0,-1.0,2.0
6,6,6,6,0.12,-1.34078,1.93609,0,-1.0,2.0
7,7,7,7,0.14,-1.32259,1.91844,0,-1.0,2.0
8,8,8,8,0.16,-1.30702,1.89424,0,-1.0,2.0
9,9,9,9,0.18,-1.29023,1.87302,0,-1.0,2.0


create a dataframe object from the csv file

Next few parts, we will be making our data in a usable format for feeding into our dataset

In [13]:
def list_generator(new_dataframe,user_num):
    user_x = new_dataframe[new_dataframe.ID==user_num]['modified_x'].to_list()
    user_y = new_dataframe[new_dataframe.ID==user_num]['modified_y'].to_list()
    user_timestamp = new_dataframe[new_dataframe.ID==user_num]['timestamp'].to_list()
    l = len(new_dataframe[new_dataframe.ID==user_num])
    user_path = [(user_x[i],user_y[i],user_timestamp[i])for i in range(l)]
    return user_path 

For 50 infected users, we will create a list of lists where each list is the path of one infected patient

In [12]:
querier_path = list_generator(df,0)
infected_list = []
for i in range(1,134):
    infected_list.append(list_generator(df,i))

In [14]:
min_x = -5
max_x = 5
min_y = -8
max_y = 9

In [26]:
def position_determiner(x, y):
    '''Given a latitude, longitude co-ordinate, we have to find the small universe it belongs to.'''
    k = math.floor((x-min_x)/3)
    j = math.floor((y-min_y)/3)
    return (k,j)
    

In [27]:
def path_divider(path):
    '''given a path (a list of tuples, (lat, lon, timestamp)), divides it into zones.
    returns: a dictionary zone->[{timestamp->(lat,lon, timestamp)},[timestamp_list]]'''
    track_dict = {}
    for elt in path:
        temp_val = position_determiner(elt[0], elt[1])
        if temp_val not in track_dict:
            track_dict[temp_val]=[{},[]]
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
        else:
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
    return track_dict


In [28]:
class Querier():
    def __init__(self,true_path):
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.intersection = []

    def filter_divider(self):
        '''returns zone->timestamp_list dictionary.'''
        self.filtered_dictionary = path_divider(self.true_path)     
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict

    def checker(self, freq_dict, zone):
        '''given the dictionary, checks the intersection with its elements, in other words, for each of its elements in this zone,
        it checks whether the frequency in the dictionary is greater than 1.'''
        timestamp_list = self.filtered_dictionary[zone][1]
        new_all_list = [zone+(timestamp,) for timestamp in timestamp_list]
        for elt in new_all_list:
            if elt in freq_dict:
                if math.floor(freq_dict[elt])>=1:
                    self.intersection.append(elt)
                
    def announcer(self):
        '''returns the self.intersection list.'''
        result_list=[]
        for elt in self.intersection:
            result_list.append(elt[2])
        return result_list


In [29]:
class Infected():
    def __init__(self, true_path,width, depth,epsilon,seed):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.seed = seed
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.new_level2_filter = {}
    #Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
    #we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
    #and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
    #result vector. 

    def Client_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(self.seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 

    def send_flipped_bit_vector(self,zone):
        result_list = []
        for elt in self.new_level2_filter[zone]:
            result_list.append(self.Client_Side(elt))
        return result_list

    def filter_divider(self):
        '''returns: zone->timestamp_list.'''
        self.filtered_dictionary = path_divider(self.true_path)
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict
        
    def filter2(self,filtered_dict_from_server):
        '''filtered_dict_from_server, a dictionary with zone->timestamps.
        modifies: a new_level2_filter list, and new_level2_filter_list[zone] is a list of (lat, lon, timestamp).'''
        for zone in filtered_dict_from_server:
            self.new_level2_filter[zone]=[]
            for elt in filtered_dict_from_server[zone]:
                self.new_level2_filter[zone].append(self.filtered_dictionary[zone][0][elt])

In [30]:
class Server():
    def __init__(self, width, depth, epsilon, seed):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.seed = seed 
        self.zone_list=[]
        self.new_filtered_list = []

    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

    def frequency_counter(self, Sketch_Matrix, total_path_length, universe_list):
        ''' we get a sketch matrix and we try to get the frequencies for all elements in the true_path.'''
        frequency_dict = {}
        for d in universe_list:
            n = total_path_length
            frac1 = self.width/(self.width-1)
            frac2 = n/self.width
            row_sum = 0
            for i in range(self.depth):
                hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
                int_hex = int(hex_num[:5],16)
                index = int_hex % self.width
                row_sum = row_sum + Sketch_Matrix[i,index]
            avg_row_sum = row_sum/self.depth 
            subtraction = avg_row_sum - frac2 
            assumed_freq = frac1*subtraction 
            frequency_dict[d]=assumed_freq
        return frequency_dict

    def checker(self,querier_filtered_path, infected_patients_filtered_dict_list):
        '''gets zone-> timestamps and returns list of filtered zone-> set(timestamps) dictionaries.'''
        l = len(infected_patients_filtered_dict_list)
        for i in range(l):
            temp_dict={}
            for zone in infected_patients_filtered_dict_list[i]:
                if zone in querier_filtered_path:
                    temp_dict[zone] = set(infected_patients_filtered_dict_list[i][zone]) & set(querier_filtered_path[zone])
            self.new_filtered_list.append(temp_dict)#For all infected patients, it maintains a dictionary, mapping zone to timestamps.
        
        return self.new_filtered_list
    
    def universe_generate(self, zone, timestamp_list):
        '''generate the universe list with the timestamps.'''
        x = zone[0]
        y = zone[1]
        x_list = []
        y_list = []
        universe_list = []
        for i in np.arange(x, x+4, 1):
            for j in np.arange(y, y+4, 1):
                for time in timestamp_list:
                    universe_list.append((i, j, time))
        return universe_list

    def bit_flip_caller(self, infected_patients, querier, run_number):
        '''Will call the flip bit method on all zones of each infected patient.'''
        zone_universe_list = set()
        l = len(self.new_filtered_list)
        for i in range(l):
            zone_universe_list|=set(self.new_filtered_list[i].keys())
        big_timestamp_dictionary = {e:set() for e in zone_universe_list}
        big_tracker_dictionary = {e:[] for e in zone_universe_list}#zone-> infected person who is in that zone
        #generating zone->timestamps for all infected patient dictionary.
        for zone in zone_universe_list:
            for i in range(l):
                if zone in self.new_filtered_list[i]:
                    big_timestamp_dictionary[zone]|=set(self.new_filtered_list[i][zone])
                    big_tracker_dictionary[zone].append(i)
        for zone in zone_universe_list:
            sketch_matrix = np.zeros((self.depth, self.width))
            for i in range(run_number):
                big_vect_list = []
                #call a function which does some calculation
                for j in big_tracker_dictionary[zone]:
                    big_vect_list+=infected_patients[j].send_flipped_bit_vector(zone)
                M = self.Compute_Sketch_Matrix(big_vect_list)
                for row in range(self.depth):
                    for col in range(self.width):
                        sketch_matrix[row, col]=(i*sketch_matrix[row, col]+M[row, col])/(i+1)
            universe_list = self.universe_generate(zone, big_timestamp_dictionary[zone])   
            n = len(big_vect_list)
            freq_dict = self.frequency_counter(sketch_matrix, n, universe_list)
            querier.checker(freq_dict, zone)
                

In [ ]:
#in the new mode, we invoke this method and the server receives this flipped bit vectors, as we want to average the 
#bit flipping thing, we run this process a mentioned number of times. and make compute sketch matrix.

In [31]:
def driver_code(width, depth, seed, epsilon, run_number):
    '''1. Create instances of querier, server and infected patients.
       2. Call the filter method on all the participants in the server, so all the querier and the infectd patients filter out their paths 
          in multiple universes. This function will output a dictionary kepys mapping to 2 lists, one list is the co-ordinates in that universe and another list is the timestamp.
       3. Now send this filtered timestamp and zone list to the server. Form can be (zone code->timestamps in that zone)
       4. Taking the querier's timestamp against the infected patient's timestamp, server will level2 filtering and send similar form of 
          dictionary to all the infected patients and the querier, so basically finding the timestamp intersection between querier and infected patient pair
          and server is keeping the lists for his calculation.
       5. The participants, upon receiving the information, filters out the information further and start processing data from their end, aka, 
          calls the server caller method.(So in our implementation, we will be processing all infected patients one by one)
       6. The server caller method, is a common method which will be used in other parts too, when this method is called from some 
          infected patient, it asks for flipped vectors a mentioned number of times from the infected patient and upon receiving it every time,
          the server does the compute sketch matrix operation and contnues averaging it and finally gets the aggregated sketch matrix, as the server knows which 
          infected patient and from which zone, it was being called, he finally calls freq generator with this information.(Zone code, sketch matrix and server has saved the timestamp info
          for generating universe list)
       7. Thus for each caller method, server keeps generating frequency histogram and keeps aggregating it and will finally send it to the querier   
    '''
    querier = Querier(querier_path)
    infected_patients = []
    for infected in infected_list:
        infected_patients.append(Infected(infected, width, depth, epsilon, seed))
    server_instance = Server(width, depth, epsilon, seed)
    infected_patients_filtered_dict_list=[]
    for e in infected_patients:
        infected_patients_filtered_dict_list.append(e.filter_divider())
    querier_filter = querier.filter_divider()
    new_filtered_list = server_instance.checker(querier_filter, infected_patients_filtered_dict_list)
    l = len(new_filtered_list)
    for i in range(l):
        infected_patients[i].filter2(new_filtered_list[i])
    server_instance.bit_flip_caller(infected_patients, querier, run_number)
    return querier.announcer()



In [32]:
   
seed = [172,  55, 177, 191, 199,  23,  82, 126,  83,  21,   6,  69, 122,
            95,  81, 194,  80,  87, 188, 159, 107,  66,   3,  38,  52,  32,
        140,  44, 185,  20, 154,  36, 186,  99, 149, 100,  53,  58,  96,
        160, 108,  46, 150, 179,   2, 142, 151,  76,  65,  72,  59, 129,
            51,  11, 193,  79, 105,  42,  77, 104,  25, 189,  35, 125,  61,
        164, 112, 190, 106,  29,  27,  33,   9, 184,  91,  26,  93, 130,
            39, 187, 174,  88, 178, 168,  63, 198, 169, 157,  10, 111, 196,
            73,  12, 115,  78, 162,  64, 147,  54, 103]
width = 150
depth = 100

In [33]:
ground_truth_timestamp_coordinates={0.0, 1.46, 1.5, 2.5, 4.2, 4.22, 4.24, 4.26, 4.28, 4.3, 4.32, 5.04, 6.0, 7.36, 8.6, 8.62, 3.5, 3.0, 2.0, 4.5, 4.0, 5.0, 7.0, 8.5, 4.72, 5.22, 5.72, 7.72, 0.42, 8.72, 8.44, 4.44, 4.94, 5.44, 5.94, 6.44, 7.44, 0.38, 0.84, 2.66, 2.16, 3.66, 3.16, 4.16, 4.66, 5.66, 6.16, 7.16, 7.66, 8.66, 2.52, 2.02, 3.52, 3.02, 1.38, 2.88, 3.38, 3.88, 4.38, 4.88, 5.38, 5.88, 6.38, 7.38, 0.3, 0.34, 4.1, 4.6, 5.6, 6.1, 6.6, 8.88, 8.38, 0.72, 1.34, 0.26, 0.76, 0.8, 1.3, 4.82, 5.32, 5.82, 6.32, 8.82, 8.54, 0.22, 2.54, 2.04, 3.54, 3.04, 4.04, 4.54, 5.54, 6.04, 7.04, 1.4, 2.9, 3.9, 0.18, 2.76, 2.26, 3.76, 3.26, 4.76, 5.26, 5.76, 6.26, 7.76, 8.76, 0.64, 0.68, 0.14, 4.48, 4.98, 5.48, 5.98, 6.98, 8.7, 8.48, 0.1, 2.7, 2.2, 3.7, 3.2, 4.7, 5.2, 5.7, 6.2, 7.2, 7.7, 2.56, 2.06, 3.56, 3.06, 0.06, 1.42, 2.42, 2.92, 3.42, 3.92, 4.42, 4.92, 5.42, 5.92, 6.42, 7.42, 8.92, 8.42, 2.78, 2.28, 3.78, 3.28, 4.64, 4.14, 0.02, 6.14, 7.64, 1.52, 1.48, 2.1, 2.68, 2.86, 2.18, 3.36, 3.68, 3.86, 3.18, 2.4, 3.4, 8.64, 8.36, 8.86, 4.36, 4.86, 5.36, 5.86, 6.36, 2.72, 2.22, 3.72, 3.22, 2.58, 2.08, 3.58, 3.08, 4.08, 4.58, 5.58, 6.08, 6.58, 1.44, 8.58, 2.44, 2.94, 3.44, 3.94, 2.8, 2.3, 3.3, 3.8, 4.8, 5.3, 5.8, 6.3, 0.36, 0.82, 0.4, 4.52, 4.02, 5.02, 5.52, 6.02, 7.02, 8.52, 8.8, 0.32, 2.74, 2.24, 3.74, 3.24, 4.74, 5.24, 5.74, 6.24, 7.74, 8.74, 2.6, 3.6, 3.1, 2.46, 2.96, 3.46, 3.96, 4.46, 4.96, 5.46, 5.96, 2.82, 3.32, 3.82, 0.7, 1.36, 0.24, 0.74, 0.28, 0.78, 1.28, 1.32, 4.68, 4.18, 5.68, 6.18, 7.18, 7.68, 8.68, 8.46, 0.2, 4.4, 4.9, 5.4, 5.9, 6.4, 7.4, 8.9, 8.4, 0.16, 2.62, 2.12, 3.62, 3.12, 4.12, 4.62, 6.12, 6.62, 7.62, 1.98, 2.48, 2.98, 3.48, 3.98, 0.66, 0.12, 0.62, 2.84, 3.34, 3.84, 4.34, 4.84, 5.34, 5.84, 6.34, 8.84, 8.34, 4.06, 4.56, 5.56, 6.06, 0.08, 6.56, 8.78, 8.56, 0.04, 4.78, 5.28, 5.78, 6.28, 7.78, 2.64, 2.14, 3.64, 3.14}

In [34]:
def check(result_list):
    check_set = set(result_list)
    true,false = 0,0
    for elt in check_set:
        if elt in ground_truth_timestamp_coordinates:
            true+=1
        else:
            false+=1
    return true, false


In [39]:
epsilon_list = [2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
run_number = [1]
final_result_dictionary1_list = []
for i in range(1):
    final_result_dictionary1 = {}
    for eps in epsilon_list:
        for run in run_number:
            check_list = driver_code(width, depth,seed, eps, run)
            final_result_dictionary1[(eps,run)] = check(check_list)
    final_result_dictionary1_list.append(final_result_dictionary1)
    print(final_result_dictionary1)
final_result_dictionary1_list

{(2.5, 1): (148, 72), (3, 1): (150, 67), (3.5, 1): (158, 71), (4, 1): (156, 83), (4.5, 1): (140, 81), (5, 1): (138, 57), (5.5, 1): (121, 61), (6, 1): (132, 52), (6.5, 1): (131, 72), (7, 1): (135, 57)}


[{(2.5, 1): (148, 72),
  (3, 1): (150, 67),
  (3.5, 1): (158, 71),
  (4, 1): (156, 83),
  (4.5, 1): (140, 81),
  (5, 1): (138, 57),
  (5.5, 1): (121, 61),
  (6, 1): (132, 52),
  (6.5, 1): (131, 72),
  (7, 1): (135, 57)}]

In [40]:
epsilon_list = [2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
epsilon_list5 = []
epsilon_list10 = []
for eps in epsilon_list:
    epsilon_list5.append(eps/5)
    epsilon_list10.append(eps/10)

In [ ]:
width = 150
depth = 100
run_number = [5]
final_result_dictionary5_list = []
for i in range(10):
    final_result_dictionary5 = {}
    for eps in epsilon_list5:
        for run in run_number:
            check_list = driver_code(width, depth,seed, eps, run)
            final_result_dictionary5[(eps,run)] = check(check_list)
    final_result_dictionary5_list.append(final_result_dictionary5)
    print(final_result_dictionary5)
final_result_dictionary5_list

In [41]:
width = 150
depth = 100
run_number = [10]
final_result_dictionary10_list = []
for i in range(1):
    final_result_dictionary10 = {}
    for eps in epsilon_list10:
        for run in run_number:
            check_list = driver_code(width, depth,seed, eps, run)
            final_result_dictionary10[(eps,run)] = check(check_list)
    final_result_dictionary10_list.append(final_result_dictionary10)
    print(final_result_dictionary10)
final_result_dictionary10_list    

{(0.25, 10): (156, 77), (0.3, 10): (151, 82), (0.35, 10): (152, 80), (0.4, 10): (137, 68), (0.45, 10): (157, 78), (0.5, 10): (143, 78), (0.55, 10): (135, 72), (0.6, 10): (163, 72), (0.65, 10): (141, 72), (0.7, 10): (174, 72)}


[{(0.25, 10): (156, 77),
  (0.3, 10): (151, 82),
  (0.35, 10): (152, 80),
  (0.4, 10): (137, 68),
  (0.45, 10): (157, 78),
  (0.5, 10): (143, 78),
  (0.55, 10): (135, 72),
  (0.6, 10): (163, 72),
  (0.65, 10): (141, 72),
  (0.7, 10): (174, 72)}]